# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
from collections import Counter
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    wordCounter = Counter(text)
    sortedWords = sorted(wordCounter, key=wordCounter.get, reverse=True)
    vocab_to_int = {word: i for i, word in enumerate(sortedWords)}
    int_to_vocab = {i: word for i, word in enumerate(sortedWords)}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Instructions for updating:
Use the retry module or similar alternatives.
Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {
        '.': '<%period%>',
        ',': '<%comma%>',
        '"': '<%quotation_mark%>',
        ';': '<%semicolon%>',
        '!': '<%exclamation_mark%>',
        '?': '<%question_mark%>',
        '(': '<%left_parentheses%>',
        ')': '<%right_parentheses%>',
        '--': '<%dash%>',
        '\n': '<%return%>'
    }
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.0
Default GPU Device: /device:GPU:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input_ = tf.placeholder(tf.int32, [None, None], name="input")
    targets_ = tf.placeholder(tf.int32, [None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    return input_, targets_, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm])
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name="initial_state")
    
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_normal([vocab_size, embed_dim], -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name="final_state")
    
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [35]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    n_batches = len(int_text) // (batch_size * seq_length)
    batch_words = n_batches * batch_size * seq_length
    x = np.array(int_text[:batch_words])
    x_batches = np.split(x.reshape(batch_size, -1), n_batches, 1)
    
    y = np.array(int_text[1:batch_words] + [int_text[0]])
    y_batches = np.split(y.reshape(batch_size, -1), n_batches, 1)
    
    return np.array(list(zip(x_batches, y_batches)))

# print(get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2))
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [36]:
# Number of Epochs
num_epochs = 20
# Batch Size
batch_size = 30
# RNN Size
rnn_size = 50
# Embedding Dimension Size
embed_dim = 4
# Sequence Length
seq_length = 5
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [38]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/460   train_loss = 8.823
Epoch   0 Batch    5/460   train_loss = 8.231
Epoch   0 Batch   10/460   train_loss = 6.965
Epoch   0 Batch   15/460   train_loss = 7.013
Epoch   0 Batch   20/460   train_loss = 6.323
Epoch   0 Batch   25/460   train_loss = 6.555
Epoch   0 Batch   30/460   train_loss = 6.608
Epoch   0 Batch   35/460   train_loss = 6.362
Epoch   0 Batch   40/460   train_loss = 6.189
Epoch   0 Batch   45/460   train_loss = 5.864
Epoch   0 Batch   50/460   train_loss = 5.791
Epoch   0 Batch   55/460   train_loss = 6.325
Epoch   0 Batch   60/460   train_loss = 6.562
Epoch   0 Batch   65/460   train_loss = 5.910
Epoch   0 Batch   70/460   train_loss = 6.405
Epoch   0 Batch   75/460   train_loss = 6.025
Epoch   0 Batch   80/460   train_loss = 5.618
Epoch   0 Batch   85/460   train_loss = 6.058
Epoch   0 Batch   90/460   train_loss = 6.084
Epoch   0 Batch   95/460   train_loss = 6.170
Epoch   0 Batch  100/460   train_loss = 5.953
Epoch   0 Batch  105/460   train_l

Epoch   1 Batch  440/460   train_loss = 5.456
Epoch   1 Batch  445/460   train_loss = 4.728
Epoch   1 Batch  450/460   train_loss = 5.326
Epoch   1 Batch  455/460   train_loss = 4.826
Epoch   2 Batch    0/460   train_loss = 4.870
Epoch   2 Batch    5/460   train_loss = 5.137
Epoch   2 Batch   10/460   train_loss = 5.187
Epoch   2 Batch   15/460   train_loss = 4.595
Epoch   2 Batch   20/460   train_loss = 4.367
Epoch   2 Batch   25/460   train_loss = 4.593
Epoch   2 Batch   30/460   train_loss = 5.100
Epoch   2 Batch   35/460   train_loss = 4.946
Epoch   2 Batch   40/460   train_loss = 4.886
Epoch   2 Batch   45/460   train_loss = 4.368
Epoch   2 Batch   50/460   train_loss = 4.451
Epoch   2 Batch   55/460   train_loss = 5.105
Epoch   2 Batch   60/460   train_loss = 5.259
Epoch   2 Batch   65/460   train_loss = 4.696
Epoch   2 Batch   70/460   train_loss = 5.138
Epoch   2 Batch   75/460   train_loss = 4.631
Epoch   2 Batch   80/460   train_loss = 4.387
Epoch   2 Batch   85/460   train_l

Epoch   3 Batch  430/460   train_loss = 4.329
Epoch   3 Batch  435/460   train_loss = 4.457
Epoch   3 Batch  440/460   train_loss = 4.789
Epoch   3 Batch  445/460   train_loss = 4.255
Epoch   3 Batch  450/460   train_loss = 4.741
Epoch   3 Batch  455/460   train_loss = 4.415
Epoch   4 Batch    0/460   train_loss = 4.424
Epoch   4 Batch    5/460   train_loss = 4.602
Epoch   4 Batch   10/460   train_loss = 4.758
Epoch   4 Batch   15/460   train_loss = 4.133
Epoch   4 Batch   20/460   train_loss = 4.011
Epoch   4 Batch   25/460   train_loss = 4.181
Epoch   4 Batch   30/460   train_loss = 4.675
Epoch   4 Batch   35/460   train_loss = 4.525
Epoch   4 Batch   40/460   train_loss = 4.471
Epoch   4 Batch   45/460   train_loss = 3.977
Epoch   4 Batch   50/460   train_loss = 4.090
Epoch   4 Batch   55/460   train_loss = 4.598
Epoch   4 Batch   60/460   train_loss = 4.798
Epoch   4 Batch   65/460   train_loss = 4.314
Epoch   4 Batch   70/460   train_loss = 4.700
Epoch   4 Batch   75/460   train_l

Epoch   5 Batch  420/460   train_loss = 3.926
Epoch   5 Batch  425/460   train_loss = 3.579
Epoch   5 Batch  430/460   train_loss = 4.001
Epoch   5 Batch  435/460   train_loss = 4.069
Epoch   5 Batch  440/460   train_loss = 4.271
Epoch   5 Batch  445/460   train_loss = 3.925
Epoch   5 Batch  450/460   train_loss = 4.307
Epoch   5 Batch  455/460   train_loss = 4.101
Epoch   6 Batch    0/460   train_loss = 4.110
Epoch   6 Batch    5/460   train_loss = 4.194
Epoch   6 Batch   10/460   train_loss = 4.459
Epoch   6 Batch   15/460   train_loss = 3.900
Epoch   6 Batch   20/460   train_loss = 3.745
Epoch   6 Batch   25/460   train_loss = 3.915
Epoch   6 Batch   30/460   train_loss = 4.373
Epoch   6 Batch   35/460   train_loss = 4.207
Epoch   6 Batch   40/460   train_loss = 4.176
Epoch   6 Batch   45/460   train_loss = 3.696
Epoch   6 Batch   50/460   train_loss = 3.858
Epoch   6 Batch   55/460   train_loss = 4.214
Epoch   6 Batch   60/460   train_loss = 4.483
Epoch   6 Batch   65/460   train_l

Epoch   7 Batch  410/460   train_loss = 3.818
Epoch   7 Batch  415/460   train_loss = 3.572
Epoch   7 Batch  420/460   train_loss = 3.834
Epoch   7 Batch  425/460   train_loss = 3.432
Epoch   7 Batch  430/460   train_loss = 3.855
Epoch   7 Batch  435/460   train_loss = 3.940
Epoch   7 Batch  440/460   train_loss = 4.000
Epoch   7 Batch  445/460   train_loss = 3.863
Epoch   7 Batch  450/460   train_loss = 4.032
Epoch   7 Batch  455/460   train_loss = 3.975
Epoch   8 Batch    0/460   train_loss = 3.799
Epoch   8 Batch    5/460   train_loss = 3.833
Epoch   8 Batch   10/460   train_loss = 4.304
Epoch   8 Batch   15/460   train_loss = 3.674
Epoch   8 Batch   20/460   train_loss = 3.543
Epoch   8 Batch   25/460   train_loss = 3.681
Epoch   8 Batch   30/460   train_loss = 4.198
Epoch   8 Batch   35/460   train_loss = 3.945
Epoch   8 Batch   40/460   train_loss = 3.957
Epoch   8 Batch   45/460   train_loss = 3.568
Epoch   8 Batch   50/460   train_loss = 3.605
Epoch   8 Batch   55/460   train_l

...

Epoch  11 Batch  390/460   train_loss = 3.846
Epoch  11 Batch  395/460   train_loss = 3.791
Epoch  11 Batch  400/460   train_loss = 3.743
Epoch  11 Batch  405/460   train_loss = 3.996
Epoch  11 Batch  410/460   train_loss = 3.574
Epoch  11 Batch  415/460   train_loss = 3.502
Epoch  11 Batch  420/460   train_loss = 3.575
Epoch  11 Batch  425/460   train_loss = 3.261
Epoch  11 Batch  430/460   train_loss = 3.723
Epoch  11 Batch  435/460   train_loss = 3.657
Epoch  11 Batch  440/460   train_loss = 3.910
Epoch  11 Batch  445/460   train_loss = 3.755
Epoch  11 Batch  450/460   train_loss = 3.797
Epoch  11 Batch  455/460   train_loss = 3.755
Epoch  12 Batch    0/460   train_loss = 3.597
Epoch  12 Batch    5/460   train_loss = 3.610
Epoch  12 Batch   10/460   train_loss = 3.947
Epoch  12 Batch   15/460   train_loss = 3.411
Epoch  12 Batch   20/460   train_loss = 3.353
Epoch  12 Batch   25/460   train_loss = 3.390
Epoch  12 Batch   30/460   train_loss = 3.938
Epoch  12 Batch   35/460   train_l

Epoch  13 Batch  375/460   train_loss = 3.550
Epoch  13 Batch  380/460   train_loss = 3.290
Epoch  13 Batch  385/460   train_loss = 3.205
Epoch  13 Batch  390/460   train_loss = 3.769
Epoch  13 Batch  395/460   train_loss = 3.751
Epoch  13 Batch  400/460   train_loss = 3.666
Epoch  13 Batch  405/460   train_loss = 3.872
Epoch  13 Batch  410/460   train_loss = 3.539
Epoch  13 Batch  415/460   train_loss = 3.438
Epoch  13 Batch  420/460   train_loss = 3.461
Epoch  13 Batch  425/460   train_loss = 3.202
Epoch  13 Batch  430/460   train_loss = 3.617
Epoch  13 Batch  435/460   train_loss = 3.523
Epoch  13 Batch  440/460   train_loss = 3.793
Epoch  13 Batch  445/460   train_loss = 3.600
Epoch  13 Batch  450/460   train_loss = 3.736
Epoch  13 Batch  455/460   train_loss = 3.687
Epoch  14 Batch    0/460   train_loss = 3.474
Epoch  14 Batch    5/460   train_loss = 3.532
Epoch  14 Batch   10/460   train_loss = 3.824
Epoch  14 Batch   15/460   train_loss = 3.321
Epoch  14 Batch   20/460   train_l

Epoch  15 Batch  365/460   train_loss = 3.488
Epoch  15 Batch  370/460   train_loss = 3.514
Epoch  15 Batch  375/460   train_loss = 3.506
Epoch  15 Batch  380/460   train_loss = 3.230
Epoch  15 Batch  385/460   train_loss = 3.161
Epoch  15 Batch  390/460   train_loss = 3.729
Epoch  15 Batch  395/460   train_loss = 3.702
Epoch  15 Batch  400/460   train_loss = 3.502
Epoch  15 Batch  405/460   train_loss = 3.783
Epoch  15 Batch  410/460   train_loss = 3.455
Epoch  15 Batch  415/460   train_loss = 3.360
Epoch  15 Batch  420/460   train_loss = 3.383
Epoch  15 Batch  425/460   train_loss = 3.147
Epoch  15 Batch  430/460   train_loss = 3.531
Epoch  15 Batch  435/460   train_loss = 3.488
Epoch  15 Batch  440/460   train_loss = 3.634
Epoch  15 Batch  445/460   train_loss = 3.493
Epoch  15 Batch  450/460   train_loss = 3.661
Epoch  15 Batch  455/460   train_loss = 3.598
Epoch  16 Batch    0/460   train_loss = 3.384
Epoch  16 Batch    5/460   train_loss = 3.459
Epoch  16 Batch   10/460   train_l

Epoch  17 Batch  355/460   train_loss = 3.806
Epoch  17 Batch  360/460   train_loss = 3.691
Epoch  17 Batch  365/460   train_loss = 3.427
Epoch  17 Batch  370/460   train_loss = 3.490
Epoch  17 Batch  375/460   train_loss = 3.449
Epoch  17 Batch  380/460   train_loss = 3.215
Epoch  17 Batch  385/460   train_loss = 3.137
Epoch  17 Batch  390/460   train_loss = 3.707
Epoch  17 Batch  395/460   train_loss = 3.660
Epoch  17 Batch  400/460   train_loss = 3.464
Epoch  17 Batch  405/460   train_loss = 3.724
Epoch  17 Batch  410/460   train_loss = 3.383
Epoch  17 Batch  415/460   train_loss = 3.264
Epoch  17 Batch  420/460   train_loss = 3.295
Epoch  17 Batch  425/460   train_loss = 3.124
Epoch  17 Batch  430/460   train_loss = 3.494
Epoch  17 Batch  435/460   train_loss = 3.442
Epoch  17 Batch  440/460   train_loss = 3.516
Epoch  17 Batch  445/460   train_loss = 3.502
Epoch  17 Batch  450/460   train_loss = 3.596
Epoch  17 Batch  455/460   train_loss = 3.627
Epoch  18 Batch    0/460   train_l

Epoch  19 Batch  345/460   train_loss = 3.368
Epoch  19 Batch  350/460   train_loss = 3.587
Epoch  19 Batch  355/460   train_loss = 3.712
Epoch  19 Batch  360/460   train_loss = 3.584
Epoch  19 Batch  365/460   train_loss = 3.387
Epoch  19 Batch  370/460   train_loss = 3.425
Epoch  19 Batch  375/460   train_loss = 3.334
Epoch  19 Batch  380/460   train_loss = 3.158
Epoch  19 Batch  385/460   train_loss = 3.089
Epoch  19 Batch  390/460   train_loss = 3.606
Epoch  19 Batch  395/460   train_loss = 3.627
Epoch  19 Batch  400/460   train_loss = 3.388
Epoch  19 Batch  405/460   train_loss = 3.638
Epoch  19 Batch  410/460   train_loss = 3.311
Epoch  19 Batch  415/460   train_loss = 3.210
Epoch  19 Batch  420/460   train_loss = 3.259
Epoch  19 Batch  425/460   train_loss = 3.066
Epoch  19 Batch  430/460   train_loss = 3.365
Epoch  19 Batch  435/460   train_loss = 3.398
Epoch  19 Batch  440/460   train_loss = 3.395
Epoch  19 Batch  445/460   train_loss = 3.488
Epoch  19 Batch  450/460   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [40]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [41]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [42]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_ = loaded_graph.get_tensor_by_name('input:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    prob = loaded_graph.get_tensor_by_name('probs:0')
    
    
    return input_, initial_state, final_state, prob


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [58]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    print(probabilities.reshape([-1]))
    return int_to_vocab[np.argmax(probabilities.reshape([-1]))]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

[0.1  0.8  0.05 0.05]
Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [60]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        print(probabilities)
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
[[[8.7749595e-03 7.7076064e-04 2.7546080e-02 ... 4.0712031e-08
   1.7691812e-08 8.7960847e-09]]]
[8.7749595e-03 7.7076064e-04 2.7546080e-02 ... 4.0712031e-08 1.7691812e-08
 8.7960847e-09]
[[[8.7749576e-03 7.7076064e-04 2.7546074e-02 ... 4.0711956e-08
   1.7691812e-08 8.7960847e-09]
  [4.7871159e-04 3.9157669e-07 1.2088090e-03 ... 1.5641929e-08
   3.4220787e-13 2.7685266e-07]]]


IndexError: index 1 is out of bounds for axis 0 with size 1

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.